In [1]:
import pandas as pd
import numpy as np
import scipy.io as sio
from scipy import stats

# Read in dorcs 


In [2]:
dorcs= pd.read_csv("E:/scrnaseqscript/foxa1_data/h2_data/H2_dorcs_nopseudo.csv", sep = ",")

In [3]:
chroms = [i.split('_')[0] for i in dorcs['peak']]

In [4]:
dorcs['Chr'] = chroms

In [5]:
x = dorcs['Gene'].value_counts(dropna=True)
y = dorcs['Gene'].value_counts(dropna=True).index
dorclist = []
dorcnums = []
z = x.index
for i in range(len(x)):
    if x[i] >= 8:
        dorclist.append(z[i])
        dorcnums.append(x[i])
dorcs["region"] = dorcs['Chr'] + "_" + (dorcs["Start"].astype(int) + 1).astype(str) + "_" + dorcs["End"].astype(str)
#dorclist

# Read in counts of atac peaks: Normalize the peak matrix - log normalize if not prenormalized

In [6]:
#reading in the cell by peak matrix
peaks = pd.read_csv("E:/scrnaseqscript/foxa1_data/h2_data/H2_bedpeaksfoxa1_new.bed", sep= "\t", header = None)
peaks.columns = ["chr", "start", "end"]
peaks["start"] = peaks["start"] + 1
peaks["region"] = peaks["start"].astype(str) + "_" + peaks["end"].astype(str)
peaks["total"] = peaks["chr"].astype(str) + "_" + peaks["region"].astype(str)


In [7]:
counts = sio.mmread("E:/scrnaseqscript/foxa1_data/h2_data/foxA1_peak_counts_H2.mtx")
cell_ids = pd.read_csv("E:/scrnaseqscript/foxa1_data/h2_data/H2peakcelltypes.txt", header = 0, sep = "\t")
cell_idrna = pd.read_csv("E:/scrnaseqscript/foxa1_data/h2_data/H2rnacelltypes.txt", header = 0, sep = "\t")


In [8]:
cell_ids = cell_ids['x']
cell_idrna = cell_idrna['x']
#cell_idrna = cell_ids['rna.bc']
#cell_ids = cell_ids['atac.bc']
#bedfile <- data(counts, package = "chromVAR")
counts.columns = cell_ids
counts.rows = peaks['total']

# find the dorc scores to create the dorc x cell matrix

In [9]:
#for each gene in dorclist
dorc_by_cell = []
for i in dorclist:
    #find all associated peak names from the table above,find them in the count matrix and sum their arrays into 1
    peak_names = dorcs[dorcs['Gene'] == i]['region'].tolist()
    geneind2 = [j for j, e in enumerate(counts.rows) if e in peak_names]
    corr = counts.tocsr()[geneind2, : ]
    corr = corr.toarray()
    corr = np.sum(corr, axis = 0)
    #print(corr)
    #add this to a new dorc_matrix with the same cell columns
    dorc_by_cell.append((i, corr))
    #print(i)

In [10]:
dorc_by_cell = pd.DataFrame(dorc_by_cell)
dorc_by_cell.head()

,0,1
0,Fam71a,"[39, 20, 11, 11, 22, 16, 15, 60, 36, 23, 12, 3..."
1,Krt16,"[54, 46, 44, 38, 10, 54, 20, 88, 52, 57, 12, 7..."
2,Mybbp1a,"[29, 23, 44, 6, 6, 8, 5, 25, 20, 40, 11, 21, 2..."
3,Brf2,"[40, 5, 19, 7, 2, 13, 19, 9, 20, 11, 0, 8, 21,..."
4,Krt8,"[26, 17, 17, 4, 9, 32, 13, 23, 18, 15, 13, 15,..."


In [11]:
dorc_by_cell.columns = ['gene', 'cell']
dorc_by_cell.index = dorc_by_cell['gene']
dorc_by_cell = dorc_by_cell[['cell']]

# read in rna counts

In [12]:
rna_expression = sio.mmread('E:/scrnaseqscript/foxa1_data/h2_data/foxA1_rna_counts_H2.mtx')



In [13]:
to_meta = pd.read_csv('E:/scrnaseqscript/foxa1_data/h2_data/H2_rna_features.tsv', sep = '\t', header = 0)

In [14]:
to_meta.head()

,x
1,Xkr4
2,Mrpl15
3,Lypla1
4,Tcea1
5,Rgs20


In [15]:
len(to_meta)

16446

In [16]:
len(cell_idrna)

3521

In [17]:
to_meta.columns = ['gene']

In [18]:
rna_expression.rows = to_meta['gene']
rna_expression.columns = cell_idrna

# Create dictionary for indexes of atac cells and rna cells in the dorc cell matrix

In [19]:
todic = counts.columns
todic = dict(todic)
todic = {v: k for k, v in todic.items()}
todic2 = dict(rna_expression.columns)

todic2 = {v: k for k, v in todic2.items()}

#dictionary of cells and their atac index
atac_cells = todic
#dictionary of cells and their rna index
rna_cells = todic2

# read in the topic knn

In [20]:
topics = pd.read_csv('E:/scrnaseqscript/foxa1_data/h2_data/cisTopic40H2.tsv', delimiter = '\t')
topics.head()

,H2_multiome#ACCTTGCTCTTGTCCA-1,H2_multiome#CTACTAAAGACAGGTA-1,H2_multiome#GGACATAAGCGAGCGA-1,H2_multiome#CAACTAATCAATCATG-1,H2_multiome#ATTCATGAGCGGATAA-1,H2_multiome#ATTGGTTCATTAGCCA-1,H2_multiome#TCATGAGGTTAGCGTA-1,H2_multiome#TATAACCCATCTTGAG-1,H2_multiome#AGGAGCTAGCTACTGG-1,H2_multiome#ATCCGTGAGACTTACA-1,...,H2_multiome#AACCGCTCACTCAACA-1,H2_multiome#CTCCTAATCTACCTAT-1,H2_multiome#TTGGTGAGTTCATTTG-1,H2_multiome#CCTAAGCCATTCCTCG-1,H2_multiome#ACCAGGACACCTGGTG-1,H2_multiome#ATCCTCATCTTAGGAC-1,H2_multiome#TACTGAGGTCTAGCTT-1,H2_multiome#CTTTAGTTCGTTTCTG-1,H2_multiome#AAGCTTGGTTTACCGT-1,H2_multiome#GAGCTTAGTGGTTCCC-1
Topic1,0.006659,0.009050,0.010735,0.009295,0.023565,0.000039,0.018717,0.024882,0.026413,0.025617,...,0.032947,0.027108,0.047781,0.040610,0.002503,0.005705,0.069680,0.018845,0.000714,0.038061
Topic2,0.012900,0.014235,0.019404,0.013930,0.021002,0.000372,0.016498,0.001633,0.004154,0.027107,...,0.017185,0.009244,0.009967,0.000511,0.009177,0.001701,0.020252,0.004031,0.003000,0.014924
Topic3,0.015606,0.019998,0.033047,0.020893,0.026886,0.048324,0.028907,0.031974,0.038643,0.040092,...,0.063157,0.019215,0.045725,0.024243,0.050334,0.040132,0.078833,0.042810,0.057857,0.057959
Topic4,0.024671,0.026210,0.024273,0.021265,0.003754,0.026890,0.007266,0.045835,0.037464,0.045654,...,0.009742,0.003843,0.011611,0.025471,0.018076,0.043735,0.002403,0.001416,0.004143,0.005206
Topic5,0.018242,0.016585,0.022015,0.016499,0.029282,0.029326,0.018415,0.026458,0.031490,0.014393,...,0.020687,0.015476,0.037916,0.004194,0.023637,0.019716,0.018421,0.007952,0.006429,0.047316


In [21]:
import sklearn.neighbors as sk
topicknn = sk.kneighbors_graph(topics.transpose(), mode='connectivity', n_neighbors=50)

In [22]:
topicknn.columns = cell_ids
topicknn.rows = cell_ids


In [23]:
topicknn

<3521x3521 sparse matrix of type '<class 'numpy.float64'>'
	with 176050 stored elements in Compressed Sparse Row format>

In [24]:
topicknn.rows = topicknn.rows.reset_index(drop = True)
topicknn.columns = topicknn.columns.reset_index(drop = True)

In [25]:
topicdict = {}
for i in range(len(topicknn.rows)):
    row = topicknn.tocsr()[i, :].toarray()
    indexes = []
    [indexes.append(j) for j in range(len(row[0])) if row[0][j] > 0]
    topicdict[topicknn.rows[i]] = indexes

# create rna x cell modified matrix

In [26]:
rna_expression

<16446x3521 sparse matrix of type '<class 'numpy.float64'>'
	with 14206915 stored elements in COOrdinate format>

In [27]:
len(rna_expression.rows.to_list())

16446

In [28]:
rind = [j for j, e in enumerate(rna_expression.rows.tolist()) if e in dorc_by_cell.index.tolist()]

In [29]:
#this difference should be very small or 0, else rind = [j + 1 ... 

In [30]:
len(rind)

878

In [31]:
set(dorc_by_cell.index.tolist()).difference([rna_expression.rows.tolist()[i] for i in rind])

set()

In [32]:
dind = [j for j, e in enumerate(dorc_by_cell.index.tolist()) if e in rna_expression.rows.tolist()]

In [33]:
#should be empty

In [34]:
set([dorc_by_cell.index.tolist()[i] for i in dind]).difference([rna_expression.rows.tolist()[i] for i in rind])

set()

In [35]:
dorc_by_cell = dorc_by_cell[dorc_by_cell.index.isin(set([dorc_by_cell.index.tolist()[i] for i in dind]))]

In [36]:

rna_expression = rna_expression.tocsr()[rind, : ]
rna_expression.columns = cell_idrna

In [37]:
rna_expression.rows = [to_meta['gene'].tolist()[i] for i in rind]
rna_by_cell = pd.DataFrame(rna_expression.todense())
rna_by_cell = rna_by_cell.values.tolist()

In [38]:

rna_by_cell2 = pd.DataFrame(rna_expression.rows)
rna_by_cell2.columns = ['gene']
rna_by_cell2 = rna_by_cell2.reset_index(drop=True)
rna_by_cell2['cell'] = [i for i in rna_by_cell]
rna_by_cell2.index = rna_by_cell2['gene']
rna_by_cell2 = rna_by_cell2[['cell']]
rna_by_cell2.head()

,cell
gene,
Rpl7,"[0.0, 0.0, 3.970291913552122, 0.0, 0.0, 3.7376..."
Neurl3,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
Ankrd39,"[0.0, 0.0, 0.6931471805599453, 0.0, 0.0, 0.0, ..."
Tmem131,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.6931471805599453, ..."
Slc9a2,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [39]:
#dorc_by_cell = pd.read_csv('dorc_by_cell.csv')

In [40]:
set(rna_by_cell2.index.tolist()).difference(set(dorc_by_cell.index.tolist()))

set()

In [41]:
set(dorc_by_cell.index.tolist()).difference(set(rna_by_cell2.index.tolist()))

set()

# smooth functions

In [42]:
def Avg(array, knn):
    toreturn = []
    for i in range(len(array)):
        val = topicknn.rows[i]
        inds = knn.get(val)
        total = int(array[i])
        for num in inds:
            total += array[num]
        total = (total/(1 + len(inds)) )
        toreturn.append(total)
    return toreturn

In [43]:
def smooth(dataframe, knn):
    count = 0
    #dataframe is a dorc x cell matrix or a rna x cell matrix
    #make sure if rna x cell matrix, we only use the dorc rnas 
    print(len(dataframe.index))
    newmat = []
    for key, value in dataframe.iterrows():
        rep = value[0]
        newrep = Avg(rep, knn)
        newmat.append((key, newrep))    
        
        #genename = key
        #geneind2 = [j for j, e in enumerate(tempy) if i == e]
        #corr = counts.tocsr()[geneind2, : ]
        #corr = corr.toarray()
        if count % 200 == 0:
            print(len(newmat))
        count += 1
    return(newmat)

# normalize our dorc matrix by peak counts - dont do this



In [ ]:
#curr = [0 for i in range(len(dorc_by_cell.iloc[0][0]))]
#for i in dorc_by_cell['cell']:
#    zipped_lists = zip(i, curr)
#    curr = [x + y for (x, y) in zipped_lists]

In [ ]:
#d2 = []
#for index, row in dorc_by_cell.iterrows():
#    zipped_lists = zip(row['cell'], curr)
#    tp = [np.log(((x / y)*10000)+1) for (x, y) in zipped_lists]
#    d2.append(tp)

#d2 = pd.Series(d2)
#d2.index = dorc_by_cell.index
#dorc_by_cell['cell'] = d2
#dorc_by_cell = dorc_by_cell[['cell']]
#dorc_by_cell.head()

# smooth the dorc scores

In [44]:
#smooth the dorc scores
smoothed_dorc = smooth(dorc_by_cell, topicdict)
#smooth the rna scores

878
1
201
401
601
801


# smooth the rna scores

In [45]:
smoothed_rna = smooth(rna_by_cell2, topicdict)

878
1
201
401
601
801


# get dorc scores and rna in usable formats

In [46]:
smoothed_rna = pd.DataFrame(smoothed_rna)
smoothed_rna.columns = ['gene', 'cell']
smoothed_dorc = pd.DataFrame(smoothed_dorc)
smoothed_dorc.columns = ['gene', 'cell']
revdorc = pd.DataFrame(cell_ids)

In [47]:
#was trying with more identified dorcs >= 10

In [48]:
matsmoothvals = smoothed_dorc.cell.apply(pd.Series)

In [49]:
matsmoothvals.index = dorc_by_cell.index
matsmoothvals = matsmoothvals.reindex(dorc_by_cell.index)

In [50]:
matsmoothvals.head()

,0,1,2,3,4,5,6,7,8,9,...,3511,3512,3513,3514,3515,3516,3517,3518,3519,3520
gene,,,,,,,,,,,,,,,,,,,,,
Fam71a,11.274510,11.803922,15.078431,10.686275,12.960784,14.607843,11.549020,20.019608,18.803922,15.117647,...,11.549020,5.980392,9.333333,6.235294,7.764706,5.764706,6.823529,2.274510,2.098039,6.196078
Krt16,20.098039,21.137255,30.431373,20.196078,20.098039,31.372549,22.039216,42.941176,41.901961,38.490196,...,12.627451,12.980392,10.921569,14.117647,13.803922,17.352941,4.450980,2.333333,2.882353,5.117647
Mybbp1a,8.078431,8.333333,13.666667,8.117647,8.254902,10.686275,8.411765,18.098039,16.098039,16.549020,...,6.901961,5.333333,4.333333,5.411765,5.647059,5.862745,3.352941,0.784314,0.784314,3.235294
Brf2,6.431373,6.745098,8.980392,6.274510,7.666667,7.627451,6.745098,9.843137,9.078431,8.078431,...,5.529412,3.882353,5.960784,3.823529,4.568627,2.705882,3.274510,0.686275,0.509804,4.196078
Krt8,8.058824,8.137255,13.725490,7.921569,8.843137,11.058824,8.647059,11.078431,12.607843,12.235294,...,7.039216,3.196078,5.000000,4.039216,5.921569,3.372549,4.313725,1.627451,2.196078,4.313725


In [52]:
matsmoothvals.to_csv('E:/scrnaseqscript/foxa1_data/h2_data/smoothed_dorc_table_H2_700.csv')

In [53]:
matsmoothvals = smoothed_rna.cell.apply(pd.Series)

In [54]:
matsmoothvals.index = rna_by_cell2.index
matsmoothvals = matsmoothvals.reindex(rna_by_cell2.index)

In [55]:
matsmoothvals.head()

,0,1,2,3,4,5,6,7,8,9,...,3511,3512,3513,3514,3515,3516,3517,3518,3519,3520
gene,,,,,,,,,,,,,,,,,,,,,
Rpl7,2.935687,3.010328,3.218136,3.001809,3.037858,3.677103,3.198629,3.597739,3.627663,3.769155,...,3.582980,3.542646,3.504519,3.596495,2.621617,3.649016,3.695713,3.778133,3.872427,3.860735
Neurl3,0.013591,0.013591,0.013591,0.013591,0.013591,0.013591,0.013591,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.013591,0.000000,0.000000,0.000000,0.000000,0.013591
Ankrd39,0.122320,0.135911,0.135911,0.122320,0.122320,0.135911,0.108729,0.135911,0.143862,0.184635,...,0.203867,0.233359,0.097447,0.200535,0.122320,0.151812,0.122320,0.054364,0.075906,0.135911
Tmem131,0.639048,0.652640,0.508534,0.646999,0.655971,0.746755,0.626855,0.829235,0.714865,0.550352,...,0.454883,0.705522,0.595413,0.563721,0.551861,0.713512,0.460054,0.614954,0.574181,0.352480
Slc9a2,0.000000,0.000000,0.013591,0.000000,0.000000,0.000000,0.000000,0.013591,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.013591,0.000000,0.000000,0.000000


In [56]:
matsmoothvals.to_csv('E:/scrnaseqscript/foxa1_data/h2_data/smoothed_rna_table_H2_700.csv')

In [57]:
cell_ids.to_csv('E:/scrnaseqscript/foxa1_data/h2_data/smoothed_dorc_columns_H2_700.csv', header = False)